In [3]:
import numpy as np
import pandas as pd
import statsmodels.api as sm

In [ ]:
df = pd.read_csv('Nat_Gas.csv')
df['Dates'] = pd.to_datetime(df['Dates'], format='%m/%d/%y')
df = df.set_index('Dates')
df = df.asfreq('ME')
df = df.sort_index()
df['Index'] = range(len(df))
df['Winter'] = df.index.month.isin([12, 1, 2]).astype(int)
df

In [ ]:
### MODEL
features = ["Index", "Winter"]
X = sm.add_constant(df[features])
Y = df["Prices"]
model = sm.OLS(Y, X).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 Prices   R-squared:                       0.746
Model:                            OLS   Adj. R-squared:                  0.735
Method:                 Least Squares   F-statistic:                     66.19
Date:               sob., 22 lut 2025   Prob (F-statistic):           3.96e-14
Time:                        18:34:49   Log-Likelihood:                -21.380
No. Observations:                  48   AIC:                             48.76
Df Residuals:                      45   BIC:                             54.37
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         10.0194      0.119     84.323      0.000       9.780      10.259
Index          0.0417      0.004     10.207      0.000       0.033       0.050
Winter         0.8296      0.131      6.345      0.000       0.566       1.093
==============================================================================
Omnibus:                        4.825   Durbin-Watson:                   1.545
Prob(Omnibus):                  0.090   Jarque-Bera (JB):                4.630
Skew:                           0.751   Prob(JB):                       0.0988
Kurtosis:                       2.754   Cond. No.                         70.9
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [61]:
# inputs
injection_dates = ["2024-01-01", "2024-04-18", "2025-02-15"] 
withdrawal_dates = ["2024-03-01", "2024-10-10", "2025-04-17"]
transport_rate = 50 # max per day
max_volume = 1000 # capacity
storage_costs = 0.16 # per day per one transport


### MID
### ================================================
injection_dates = pd.to_datetime(injection_dates)
withdrawal_dates = pd.to_datetime(withdrawal_dates)

# prices
injection_prices = list(model.predict(list(map(lambda i: [1, round((i - df.head(1).index.item()).days/30), 1 if i.month in [12, 1, 2] else 0], injection_dates))))
withdrawal_prices = list(model.predict(list(map(lambda i: [1, round((i - df.head(1).index.item()).days/30), 1 if i.month in [12, 1, 2] else 0], withdrawal_dates))))
dif = sum(withdrawal_prices) - sum(injection_prices)
dif = dif if dif > 0 else 0

# days
days = list(map(lambda i, w: (w - i).days, injection_dates, withdrawal_dates))
days = sum(days)

price = dif*transport_rate*len(injection_dates) + storage_costs*days
price = round(price, 2)

# output
print(price)

47.36
